In [ ]:
import yfinance as yf
import talib
import pandas as pd
import copy
import numpy as np
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)


def symbols_backtesting(symbol_list):
    all_trades = []
    for symbol in symbol_list:
        df = yf.Ticker(f"{symbol}.NS").history(period="1mo", interval="15m")
        df["EMA_4"] = talib.EMA(df["Close"], timeperiod=4)
        df["EMA_44"] = talib.EMA(df["Close"], timeperiod=44)


        print(df)
#         df["RSI_14"] = talib.RSI(df["Close"], timeperiod=14)
#         df["ATR_14"] = talib.ATR(df["High"], df["Low"], df["Close"], timeperiod=14)
#         df["Upper_Band"], df["Middle_Band"], df["Lower_Band"] = talib.BBANDS(df["Close"], timeperiod=20,
#                                                                              nbdevup=2,nbdevdn=2)
        data = df.dropna()
        
        trade = {"Symbol": None, "Buy/Sell": None, "Entry": None, "Entry Date": None, "Exit": None, "Exit Date": None}
        print(df)
        position = None
        
#       
        for i in range(len(df)-1):
            trade = {"Symbol": None, "Buy/Sell": None, "Entry": None, "Entry Date": None, "Exit": None, "Exit Date": None}
            position = None
            if df.iloc[i]['Close'] <= df.iloc[i]['EMA_44'] and df.iloc[i+1]['Open'] >= df.iloc[i]['EMA_44'] and df.iloc[i+1]['Open'] == df.iloc[i+1]['Low'] and df.iloc[i]['High'] <= df.iloc[i]['EMA_44'] and df.iloc[i]['Close'] >= df.iloc[i]['EMA_4'] and position != "Buy" :
#                 if trade["Symbol"] is not None:
                trade["Exit"] = df.iloc[i+1]['Close']
                trade["Exit Date"] = df.index[i+1]
#                     all_trades.append(copy.deepcopy(trade))
#                 if position is not None:
                trade["Symbol"] = symbol
                trade["Buy/Sell"] = "Buy"
                trade["Entry"] = df.iloc[i+1]['Open']
                trade["Entry Date"] =df.index[i]
            position = "Buy"
            if df.iloc[i]['Close'] >= df.iloc[i]['EMA_44'] and df.iloc[i+1]['Open'] <= df.iloc[i]['EMA_44'] and df.iloc[i]['Low'] and  df.iloc[i]['Low'] >= df.iloc[i]['EMA_44'] and position != "Sell":
#                 if trade["Symbol"] is not None:
                trade["Exit"] = df.iloc[i+1]['Close']
                trade["Exit Date"] = df.index[i+1]
#                     all_trades.append(copy.deepcopy(trade))
#                 if position is not None:
                trade["Symbol"] = symbol
                trade["Buy/Sell"] = "Sell"
                trade["Entry"] = df.iloc[i+1]['Close']
                trade["Entry Date"] = df.index[i+1]
            position = "Sell"
            all_trades.append(trade)
    return all_trades


symbol_list = [ "TATAMOTORS" , "IPCALAB"]

data = symbols_backtesting(symbol_list)
if data:
    risk_percent = 5/100
    df = pd.DataFrame(data)
    df["P/L"] = np.where(df["Buy/Sell"] == "Buy", (100*(df["Exit"] - df["Entry"])/df["Entry"])*risk_percent,
                         (100*(df["Entry"] - df["Exit"])/df["Entry"])*risk_percent)
    df = df[df["Buy/Sell"] == "Buy"].reset_index(drop=True)

    df["Probability"] = 100*(np.where(df["P/L"] > 0, 1, 0).cumsum())/(np.where(df["P/L"] != np.NaN, 1, 0).cumsum())
    df["Return"] = df["P/L"].cumsum()
    df["Drawdown"] = df["Return"] - (df["Return"].cummax().apply(lambda x: x if x > 0 else 0))

#     print(df)
else:
    print("No Trades")











In [ ]:
df

In [ ]:
import yfinance as yf
import talib
from talib import abstract 
import pandas as pd
import copy
import numpy as np
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

symbol_list = ['ADANIPORTS','APOLLOHOSP','ASIANPAINT','AXISBANK','BAJAJFINSV','BAJFINANCE','BHARTIARTL','BPCL','BRITANNIA','CIPLA','COALINDIA','DIVISLAB','DRREDDY','EICHERMOT','GRASIM','HCLTECH','HDFC','HDFCBANK','HDFCLIFE','HEROMOTOCO','HINDALCO','HINDUNILVR','ICICIBANK','INDUSINDBK','INFY','ITC','JSWSTEEL','KOTAKBANK','LT','M&M','MARUTI','NESTLEIND','NTPC','ONGC','POWERGRID','RELIANCE','SBILIFE','SBIN','SHREECEM','SUNPHARMA','TATACONSUM','TATAMOTORS','TATASTEEL','TCS','TECHM','TITAN','ULTRACEMCO','UPL','WIPRO']

def symbols_backtesting(symbol_list):
    all_trades = []
    for symbol in symbol_list:
        df = yf.Ticker(f"{symbol}.NS").history(period="3mo", interval="60m")
        df["EMA_4"] = abstract.EMA(df["Close"], timeperiod=4)
        df["EMA_44"] = abstract.EMA(df["Close"], timeperiod=44)
   
        
#       
        for i in range(len(df)-1):
            trade = {"Symbol": None, "Buy/Sell": None, "Entry": None, "Entry Date": None, "Exit": None, "Exit Date": None}
            position = None
            if df.iloc[i]['Close'] >= df.iloc[i]['EMA_44'] and df.iloc[i+1]['Open'] == df.iloc[i+1]['High'] and df.iloc[i+1]['Open'] <= df.iloc[i]['EMA_44'] and df.iloc[i+1]['Open'] <= df.iloc[i+1]['High'] and df.iloc[i]['Low'] >= df.iloc[i]['EMA_44'] and position != "Sell":
#                 if trade["Symbol"] is not None:
                trade["Exit"] = df.iloc[i+1]['Low']
                trade["Exit Date"] = df.index[i+1]
#                     all_trades.append(copy.deepcopy(trade))
#                 if position is not None:
                trade["Symbol"] = symbol
                trade["Buy/Sell"] = "Sell"
                trade["Entry"] = df.iloc[i+1]['Open']
                trade["Entry Date"] = df.index[i]
            position = "Sell"
            all_trades.append(trade)
    return all_trades

data = symbols_backtesting(symbol_list)


if data:
    risk_percent = 5/100
    df = pd.DataFrame(data)
    df["P/L"] = np.where(df["Buy/Sell"] == "Sell", (100*(df["Exit"] - df["Entry"])/df["Entry"])*risk_percent,
                         (100*(df["Entry"] - df["Exit"])/df["Entry"])*risk_percent)
    df = df[df["Buy/Sell"] == "Sell"].reset_index(drop=True)

    df["Probability"] = 100*(np.where(df["P/L"] > 0, 1, 0).cumsum())/(np.where(df["P/L"] != np.NaN, 1, 0).cumsum())
    df["Return"] = df["P/L"].cumsum()
    df["Drawdown"] = df["Return"] - (df["Return"].cummax().apply(lambda x: x if x > 0 else 0))

    print(df)
else:
    print("No Trades")











In [ ]:
df

In [1]:
%matplotlib inline 
import matplotlib.pyplot as plt
import yfinance as yf
import talib
from talib import abstract 
import pandas as pd
import copy
import numpy as np
import mplfinance as mpf

from datetime import datetime
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)


def symbols_backtesting(symbol_list):
    all_trades = []
    for symbol in symbol_list:
        df = yf.Ticker(f"{symbol}.NS").history(period="3mo", interval="60m")
        df["EMA_4"] = abstract.EMA(df["Close"], timeperiod=4)
        df["EMA_44"] = abstract.EMA(df["Close"], timeperiod=44)
        #       
        for i in range(len(df)-1):
            trade = {"Symbol": None, "Buy/Sell": None, "Entry": None, "Entry Date": None, "Exit": None, "Exit Date": None}
            position = None
            if df.iloc[i]['Close'] >= df.iloc[i]['EMA_44'] and df.iloc[i+1]['Open'] == df.iloc[i+1]['High'] and df.iloc[i+1]['Open'] <= df.iloc[i]['EMA_44'] and df.iloc[i+1]['Open'] <= df.iloc[i+1]['High'] and df.iloc[i]['Low'] >= df.iloc[i]['EMA_44'] and position != "Sell":
#                 if trade["Symbol"] is not None:
                trade["Exit"] = df.iloc[i+1]['Low']
                trade["Exit Date"] = df.index[i+1]
#                     all_trades.append(copy.deepcopy(trade))
#                 if position is not None:
                trade["Symbol"] = symbol
                trade["Buy/Sell"] = "Sell"
                trade["Entry"] = df.iloc[i+1]['Open']
                trade["Entry Date"] = df.index[i]
                point = [np.nan for i in range(len(df))]
                point[i] = trade['Entry']
                apd = mpf.make_addplot(point, type='scatter', marker='v', markersize=200)
                mpf.plot(df,type='candle',figsize=(25,18), figratio=(20,12),ema=(4,44),volume=True,tight_layout=True,style='yahoo', addplot=apd)
                
            position = "Sell"


            all_trades.append(trade)
    return all_trades


symbol_list = ['ADANIPORTS','APOLLOHOSP','ASIANPAINT','AXISBANK','BAJAJFINSV','BAJFINANCE','BHARTIARTL','BPCL','BRITANNIA','CIPLA','COALINDIA','DIVISLAB','DRREDDY','EICHERMOT','GRASIM','HCLTECH','HDFC','HDFCBANK','HDFCLIFE','HEROMOTOCO','HINDALCO','HINDUNILVR','ICICIBANK','INDUSINDBK','INFY','ITC','JSWSTEEL','KOTAKBANK','LT','M&M','MARUTI','NESTLEIND','NTPC','ONGC','POWERGRID','RELIANCE','SBILIFE','SBIN','SHREECEM','SUNPHARMA','TATACONSUM','TATAMOTORS','TATASTEEL','TCS','TECHM','TITAN','ULTRACEMCO','UPL','WIPRO']


data = symbols_backtesting(symbol_list)
if data:
    risk_percent = 5/100
    df = pd.DataFrame(data)
    df["P/L"] = np.where(df["Buy/Sell"] == "Sell", (100*(df["Exit"] - df["Entry"])/df["Entry"])*risk_percent,
                         (100*(df["Entry"] - df["Exit"])/df["Entry"])*risk_percent)
    df = df[df["Buy/Sell"] == "Sell"].reset_index(drop=True)

    df["Probability"] = 100*(np.where(df["P/L"] > 0, 1, 0).cumsum())/(np.where(df["P/L"] != np.NaN, 1, 0).cumsum())
    df["Return"] = df["P/L"].cumsum()
    df["Drawdown"] = df["Return"] - (df["Return"].cummax().apply(lambda x: x if x > 0 else 0))

    print(df)
else:
    print("No Trades")

KeyboardInterrupt: 